In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

param_df = pd.read_csv('/mnt/analysis/e17023/Adam/GADGET2/simInput/parameters.csv')
param_df = param_df[param_df['Score'] > 0]

with open('/mnt/analysis/e17023/Adam/GADGET2/simOutput/tuning_params.json', 'r') as f:
    tuning_params = json.load(f)

offset = 0
for param in tuning_params:
    if param not in ['Settings']:
        min_val = tuning_params[param][1]
        max_val = tuning_params[param][2]
        best_val = tuning_params[param][0]
        param_df[param] = param_df[param].apply(lambda x: (x - min_val) / (max_val - min_val))
        param_df[param] = np.clip(param_df[param], 0, 1) + offset
        
        plt.plot([0, len(param_df)], [offset, offset], 'k')
        
        # print best value in the middle of the plot
        plt.text(len(param_df), offset + 0.4, str(round(best_val,6)), horizontalalignment='center', verticalalignment='center')
        
        offset -= 1
        plt.plot(param_df[param], label = param)

# don't show y axis ticks or labels, legend on the left
plt.legend(loc='center left')
plt.yticks([])

# save plot
plt.savefig('/mnt/analysis/e17023/Adam/GADGET2/simOutput/params.png')

# make new plot of score over time
plt.clf()


new_best_score = []
nb_i = []
for i in range(len(param_df)):
    if i == 0:
        new_best_score.append(param_df['Score'].iloc[i])
        nb_i.append(i)
    else:
        if param_df['Score'].iloc[i] < new_best_score[-1]:
            new_best_score.append(param_df['Score'].iloc[i])
            nb_i.append(i)

new_best_score.append(new_best_score[-1])
nb_i.append(len(param_df))

plt.plot(param_df['Score'], 'o')
plt.plot(nb_i, new_best_score, label = str(new_best_score[-2]) + ' at ' + str(nb_i[-2]))

# fit an exponential to the data
from scipy.optimize import curve_fit
def func(x, a, b, c):
    x = np.array(x)
    return a * np.exp(-b * x) + c

popt, pcov = curve_fit(func, nb_i, new_best_score)
plt.plot(nb_i, func(nb_i, *popt), 'r-', label = 'fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))

plt.legend()
plt.savefig('/mnt/analysis/e17023/Adam/GADGET2/simOutput/score.png')



NameError: name 'pd' is not defined